This notebook is meant to extract features from the generated images.

# Setup environment

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
image_feature_folder = "/gdrive/MyDrive/Final Project/Data/Features"
annotations_folder = "/gdrive/MyDrive/Final Project/Data/Annotations"
generated_images_folder = "/gdrive/MyDrive/Final Project/Data/Generated Images"
real_images_folder = "/gdrive/MyDrive/Final Project/Data/Real Images"

Mounted at /gdrive


#Get pre trained models

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocessor, decode_predictions
from tensorflow.keras import Model
# from skimage import io
from skimage.transform import resize
import numpy as np
import pandas as pd
from tqdm import tqdm
from numpy import savetxt
import os
import imageio as iio

In [ ]:
pretrained_models = {}

In [ ]:
base_model = ResNet50(weights='imagenet')
resnet_model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
pretrained_models["resnet_model"] = (resnet_preprocessor, resnet_model)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocessor

In [ ]:
base_model = VGG16(weights='imagenet')
vgg16_model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
pretrained_models["vgg16_model"] = (vgg16_preprocessor, vgg16_model)

## Preprocess dataset

In [ ]:
def chunker(seq, size):
  return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
def get_image(image_path, resize_shape=(224,224)):
  image = iio.imread(image_path)
  image = resize(image, resize_shape)
  return image
def preprocess_image(images, pretrained_model_preprocessor):
  preprocess_input, model =  pretrained_model_preprocessor
  x = preprocess_input(images)
  features = model.predict(x, verbose = 0)
  return features
def save_features(features,file_name, path):
  # file_name = 'features_{}.txt'.format(index)
  savetxt(os.path.join(path,file_name), features, delimiter=',')
def preprocess_images(image_path, pretrained_model_preprocessor, features_path, batch_size = 64):
  image_paths = [ os.path.join(image_path, image_name) for image_name in os.listdir(image_path) ]
  image_path_chunks = chunker(image_paths, batch_size)
  existing_files = set(os.listdir(features_path))
  for index, image_paths in tqdm(enumerate(image_path_chunks), total = len(image_path_chunks), leave=False):
    file_name = 'features_{}.txt'.format(index)
    if file_name not in existing_files:
      images = np.array(list(map(get_image, image_paths)))
      features = preprocess_image(images, pretrained_model_preprocessor)
      save_features(features, file_name, features_path)
def extract_features(images_folders, pretrained_models):
  for image_type, folder_path in images_folders.items():
    folders = [os.path.join(folder_path, folder) for folder in os.listdir(folder_path) if "." not in folder] #This will have train and val folder locations
    for model_name, model_data in pretrained_models.items():
      features_folder = os.path.join(image_feature_folder,image_type,model_name)
      for folder in folders:
        print(folder, "->",features_folder)
        preprocess_images(folder, model_data,features_folder)


In [ ]:
images_folders = {
    "Generated" : generated_images_folder,
    "Real" : real_images_folder
}

In [ ]:
extract_features(images_folders, pretrained_models)

/gdrive/MyDrive/Final Project/Data/Generated Images/train -> /gdrive/MyDrive/Final Project/Data/Features/Generated/vgg16_model


/gdrive/MyDrive/Final Project/Data/Generated Images/val -> /gdrive/MyDrive/Final Project/Data/Features/Generated/vgg16_model


/gdrive/MyDrive/Final Project/Data/Generated Images/train -> /gdrive/MyDrive/Final Project/Data/Features/Generated/resnet_model


/gdrive/MyDrive/Final Project/Data/Generated Images/val -> /gdrive/MyDrive/Final Project/Data/Features/Generated/resnet_model
